In [ ]:
pip install transformers


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
reviews= pd.read_csv(r'/content/sample_data/reviews_vader_sentiment.csv',index_col=0)

reviews.head()

,Experience_Summary,Overall_Rating,Customer_Name,Trip_Verified,Experience_Detail,Traveller_Type,Seat_Type,Flight_Route,Flight_Date,Seat_Comfort,...,Recommended,Food_and_Beverages,Inflight_Entertainment,WiFi_and_Connectivity,Airline_Name,Year_Flown,Month_Flown,Number_of_Stops,Trip_Type,vader_sentiment
0,This has been a total nightmare,1,V Darkana,1.0,My flight was scheduled on July 4 2021 Sunda...,Family Leisure,Economy Class,Anchorage to San Francisco,Jul-21,1.0,...,no,3.0,3.0,1.0,United Airlines,2021,July,0.0,International,negative
1,beyond appalled by this experience,1,Bishop Jermaine Moore Sr.,0.0,United Airlines have successfully and repetit...,Family Leisure,Economy Class,Pittsburgh to Dubai via Newark,May-23,3.0,...,no,1.0,1.0,1.0,United Airlines,2023,May,1.0,International,negative
2,Both segments were awful,1,J Parcell,1.0,Both segments were awful. The single agent f...,Solo Leisure,Economy Class,Montrose to Denver,Jul-21,2.0,...,no,1.0,1.0,1.0,United Airlines,2021,July,0.0,International,negative
3,never fly through this airline again,1,S Laytan,1.0,We reviewed the dimensions and the sizes on ...,Family Leisure,Economy Class,Pasco to Miami,Jun-21,1.0,...,no,1.0,1.0,1.0,United Airlines,2021,June,0.0,International,positive
4,fly other airlines in the future,1,Nancy S Giannetta,1.0,So as I boarded I ask for help putting my ca...,Solo Leisure,Business Class,Chicago to Tampa,Apr-24,2.0,...,no,2.0,2.0,1.0,United Airlines,2024,April,0.0,International,positive


In [ ]:
import spacy

In [ ]:
nlp=spacy.load('en_core_web_sm')

In [ ]:
reviews['Airline_Name'].value_counts()

,count
Airline_Name,
American Airlines,1901
Spirit Airlines,1815
Frontier Airlines,1799
United Airlines,1425
Delta Airlines,1246
Jet Blue Airlines,844
South West Airlines,774
Allegiant Air Airlines,705


In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# Load the tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

# Example review



vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Sentiment: Positive
Confidence Scores: tensor([[0.0016, 0.0071, 0.9913]])


In [ ]:
def preprocess(text):
    text=nlp(text)
    filtered_text=[]
    for each in text:
        if each.is_stop or each.is_punct:
          continue
        else:
            filtered_text.append(each.lemma_)
    return ' '.join(filtered_text)

In [ ]:
preprocess('kumar is not working today')

'kumar work today'

In [ ]:
def sentiment_score(text):
  review = preprocess(text)
# Tokenize and encode the review text
  inputs = tokenizer(review, return_tensors="pt", padding=True, truncation=True,max_length=512)

# Perform inference (no gradient calculation needed)
  with torch.no_grad():
      outputs = model(**inputs)
      predictions = torch.softmax(outputs.logits, dim=-1)

  # Extract the predicted sentiment label
  predicted_class = predictions.argmax().item()
  label_mapping = {0: "Negative", 1: "Neutral", 2: "Positive"}

  return label_mapping[predicted_class]


In [ ]:
reviews['roberta_sentiment']= reviews['Experience_Detail'].apply(sentiment_score)

In [ ]:
reviews.head()

,Experience_Summary,Overall_Rating,Customer_Name,Trip_Verified,Experience_Detail,Traveller_Type,Seat_Type,Flight_Route,Flight_Date,Seat_Comfort,...,Food_and_Beverages,Inflight_Entertainment,WiFi_and_Connectivity,Airline_Name,Year_Flown,Month_Flown,Number_of_Stops,Trip_Type,vader_sentiment,roberta_sentiment
0,This has been a total nightmare,1,V Darkana,1.0,My flight was scheduled on July 4 2021 Sunda...,Family Leisure,Economy Class,Anchorage to San Francisco,Jul-21,1.0,...,3.0,3.0,1.0,United Airlines,2021,July,0.0,International,negative,Negative
1,beyond appalled by this experience,1,Bishop Jermaine Moore Sr.,0.0,United Airlines have successfully and repetit...,Family Leisure,Economy Class,Pittsburgh to Dubai via Newark,May-23,3.0,...,1.0,1.0,1.0,United Airlines,2023,May,1.0,International,negative,Negative
2,Both segments were awful,1,J Parcell,1.0,Both segments were awful. The single agent f...,Solo Leisure,Economy Class,Montrose to Denver,Jul-21,2.0,...,1.0,1.0,1.0,United Airlines,2021,July,0.0,International,negative,Negative
3,never fly through this airline again,1,S Laytan,1.0,We reviewed the dimensions and the sizes on ...,Family Leisure,Economy Class,Pasco to Miami,Jun-21,1.0,...,1.0,1.0,1.0,United Airlines,2021,June,0.0,International,positive,Neutral
4,fly other airlines in the future,1,Nancy S Giannetta,1.0,So as I boarded I ask for help putting my ca...,Solo Leisure,Business Class,Chicago to Tampa,Apr-24,2.0,...,2.0,2.0,1.0,United Airlines,2024,April,0.0,International,positive,Neutral


In [ ]:
reviews['roberta_sentiment'].value_counts()

,count
roberta_sentiment,
Negative,6237
Neutral,3428
Positive,844


In [ ]:
reviews.to_csv('reviews_vr_sentiments.csv')